In [183]:
import os
import pandas as pd
from dotenv import load_dotenv
from py2neo import Graph

load_dotenv()
password = os.getenv('DBPASS')

graph = Graph("bolt://localhost:7687", auth=("neo4j", password))

In [121]:
# not really working yet
#query = """
#MATCH (i:Issuelist {name: "Umwelt"})<-[:ABOUT]-(b:Bill:Motion)<-[:SPONSORS]-(p:Person)-[:GAVE]->(s:Speech)
#RETURN DISTINCT p, s;
#"""

#speeches1 = graph.run(query).data()
#len(speeches1)

52225

In [147]:
#speeches1[1]

{'p': Node('Person', 'Unique Person', date_birth=Date(1941, 9, 15), first_name='Ruedi', gender='m', last_name='Aeschbacher', uid=449),
 's': Node('Inconsistent Timestamp', 'Speech', 'Transcript', language_of_text='DE', length=1204, name='Aeschbacher Ruedi', order_stamp=25, role='For the committee', time_end=DateTime(2009, 4, 30, 8, 39, 39.0), time_start=DateTime(2009, 4, 30, 8, 37, 52.0), uid='166224')}

In [146]:
#query = """
#MATCH (i:Issuelist {name: "Umwelt"})<-[:ABOUT]-(b:Bill:Motion)<-[:SPONSORS]-(p:Person)-[:GAVE]->(s:Speech)
#RETURN DISTINCT b, p, s;
#"""

#speeches2 = graph.run(query).data()
#len(speeches2)

140998

In [131]:
#len(speeches2)/len(speeches1)

2.6998180947821924

In [194]:
query = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton), 
    (p)-[:ELECTED_TO]->(ch:Chamber),
    (b)-[:CONTAINS]->(d:`Chamber Debate`)
    
RETURN i.name AS issue, 
    c.name AS canton, 
    ch.name AS chamber,
    COUNT(b) AS num_bills, 
    substring(toString(d.date_end), 0, 4) AS year
    
ORDER BY issue, canton, year;
"""

results = graph.run(query).data()
df = pd.DataFrame(results)

# Display the DataFrame as a table
df

,issue,canton,chamber,num_bills,year
0,Beschäftigung und Arbeit,Aargau,Nationalrat,49,2012
1,Beschäftigung und Arbeit,Aargau,Ständerat,24,2012
2,Beschäftigung und Arbeit,Aargau,Nationalrat,18,2013
3,Beschäftigung und Arbeit,Aargau,Nationalrat,47,2014
4,Beschäftigung und Arbeit,Aargau,Ständerat,4,2014
...,...,...,...,...,...
12207,Zivilrecht,Zürich,Nationalrat,148,2021
12208,Zivilrecht,Zürich,Nationalrat,184,2022
12209,Zivilrecht,Zürich,Ständerat,36,2022
12210,Zivilrecht,Zürich,Ständerat,12,2023
